In [2]:
# !pip install langchain

### for logging traces

In [1]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

### Integrating LLM

In [7]:
# !pip install -qU langchain-groq
# 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

In [98]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain_groq import ChatGroq

model = ChatGroq(model="llama-3.3-70b-versatile", temperature=0.0)

In [99]:
# echo %GROQ_API_KEY%

In [100]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("I want to open a restaurant for Indian food. Suggest me a name."),

]

mssg = model.invoke(messages)

In [101]:
dict(mssg)

{'content': 'Here are some name suggestions for your Indian restaurant:\n\n1. **Tandoori Nights**: Inspired by the traditional Indian clay oven, tandoor, this name evokes a sense of warmth and exotic flavors.\n2. **Spice Route**: This name references the historic trade route that connected India to the rest of the world, conveying a sense of adventure and diverse flavors.\n3. **Dhaba**: A dhaba is a traditional Indian roadside eatery, and using this name can create a cozy, casual atmosphere for your restaurant.\n4. **Mumbai Masala**: This name combines the vibrant city of Mumbai with the idea of blended spices, suggesting a rich and flavorful dining experience.\n5. **Taste of India**: Simple and straightforward, this name promises customers a authentic taste of Indian cuisine.\n6. **Bollywood Bites**: This name incorporates the popular Indian film industry, Bollywood, and suggests a fun, lively atmosphere for your restaurant.\n7. **Garam**: Meaning "hot" or "warm" in Hindi, this name c

In [96]:
print(mssg.content)

Here are some name suggestions for your Indian restaurant:

1. **Tandoori Nights**: Inspired by the traditional Indian clay oven, tandoor, this name evokes a sense of warmth and exotic flavors.
2. **Spice Route**: This name references the historic trade route that connected India to the rest of the world, conveying a sense of adventure and diverse flavors.
3. **Dhaba**: A dhaba is a traditional Indian roadside eatery, and using this name can create a cozy, casual atmosphere for your restaurant.
4. **Mumbai Masala**: This name combines the vibrant city of Mumbai with the idea of blended spices, suggesting a rich and flavorful dining experience.
5. **Taste of India**: Simple and straightforward, this name promises customers a authentic taste of Indian cuisine.
6. **Bollywood Bites**: This name incorporates the popular Indian film industry, Bollywood, and suggests a fun, lively atmosphere for your restaurant.
7. **Garam**: Meaning "hot" or "warm" in Hindi, this name can create a sense of 

In [116]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables=["restaurant_type"],
    template="I want to open a restaurant for {restaurant_type}. Suggest me single name ,only name"
)
prompt_template_name.format(restaurant_type="Indian food")

'I want to open a restaurant for Indian food. Suggest me single name ,only name'

In [121]:
from langchain.chains import LLMChain
resturant_chain = LLMChain(llm=model,prompt=prompt_template_name)
resturant_name = resturant_chain.run(restaurant_type="Indian food")
print(resturant_name)

Tandoori Nights


In [118]:
## creating Sequential chain

prompt_template_name_food = PromptTemplate(
    input_variables=["resturant_name"],
    template="Please suggest some menu items for {resturant_name}."
)

food_items_chain = LLMChain(llm=model,prompt=prompt_template_name_food)


In [125]:
food_names = food_items_chain.run(resturant_name=resturant_name)
print(food_names)

Tandoori Nights sounds like a wonderful Indian restaurant. Here are some delicious menu item suggestions:

**Appetizers**

1. Vegetable Samosas - Crispy fried or baked pastries filled with spiced potatoes, peas, and onions.
2. Chicken Tikka - Marinated chicken cooked in a creamy tomato sauce, served with a side of cucumber raita.
3. Paneer Pakora - Crispy fried cheese fritters served with a side of mint chutney.
4. Papdi Chaat - A street food-style snack with boiled potatoes, chickpeas, and yogurt, topped with crispy fried bread and tamarind chutney.

**Tandoori Specialties**

1. Chicken Tandoori - Marinated chicken cooked in a clay oven, served with a side of cucumber raita and basmati rice.
2. Lamb Seekh Kebab - Minced lamb skewers cooked in a clay oven, served with a side of mint chutney.
3. Vegetable Tandoori Platter - A selection of marinated vegetables cooked in a clay oven, served with a side of cucumber raita and basmati rice.
4. Naan Bread - Leavened, butter-topped flatbread c

In [128]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [resturant_chain, food_items_chain])
response = chain.run("Indian food")

In [130]:
print(response)

Tandoori Nights sounds like a wonderful Indian restaurant. Here are some delicious menu item suggestions:

**Appetizers**

1. Vegetable Samosas - Crispy fried or baked pastries filled with spiced potatoes, peas, and onions.
2. Chicken Tikka - Marinated chicken cooked in a creamy tomato sauce, served with a side of cucumber raita.
3. Paneer Pakora - Crispy fried cheese fritters served with a side of mint chutney.
4. Papdi Chaat - A street food-style snack with boiled potatoes, chickpeas, and yogurt, topped with crispy fried bread and tamarind chutney.

**Tandoori Specialties**

1. Chicken Tandoori - Marinated chicken cooked in a clay oven, served with a side of cucumber raita and basmati rice.
2. Lamb Seekh Kebab - Minced lamb skewers cooked in a clay oven, served with a side of mint chutney.
3. Vegetable Tandoori Platter - A selection of marinated vegetables cooked in a clay oven, served with a side of cucumber raita and basmati rice.
4. Naan Bread - Leavened, butter-topped flatbread c

In [142]:
## for both ouput at the same time

model1= ChatGroq(model="llama-3.3-70b-versatile", temperature=0.5)
prompt_template_name_food = PromptTemplate(
    input_variables=["cuisine"],
    template="I want to open a restaurant for {cuisine}. Suggest me single name ,only name"
)

name_chain = LLMChain(llm=model1,prompt=prompt_template_name_food, 
                            output_key= "restaurant_name")

model2= ChatGroq(model="llama-3.3-70b-versatile", temperature=0.5)
prompt_template_name = PromptTemplate(
    input_variables=["restaurant_name"],
    template="Suggestions for menu items for {restaurant_name}."
)
food_items_chain = LLMChain(llm=model2,prompt=prompt_template_name,
                            output_key= "food_items")
print(food_items_chain.run(restaurant_name=name_chain.run(cuisine="Indian food")))

Tandoori Nights sounds like a fantastic Indian restaurant. Here are some delicious menu item suggestions to get you started:

**Appetizers**

1. Vegetable Samosas - Crispy fried or baked pastries filled with spiced potatoes, peas, and onions.
2. Chicken Tikka - Marinated chicken cooked in a creamy tomato sauce, served with a side of cucumber raita.
3. Paneer Tikka - Indian cheese marinated in spices and yogurt, grilled to perfection.
4. Papri Chaat - A flavorful street food snack made with boiled potatoes, chickpeas, and yogurt, topped with crispy papri and chutneys.
5. Onion Bhajis - Crispy fried onion fritters served with a side of mint chutney.

**Tandoori Specialties**

1. Chicken Tandoori - Classic Indian dish made by marinating chicken in yogurt and spices, then roasting it in a tandoor oven.
2. Lamb Seekh Kebab - Minced lamb mixed with spices and herbs, grilled to perfection in a tandoor oven.
3. Vegetable Tandoori - A colorful mix of marinated vegetables, such as bell peppers, 

In [143]:
from langchain.chains import SequentialChain

chain = SequentialChain(chains = [name_chain, food_items_chain],
                        input_variables=["cuisine"],
                        output_variables=["restaurant_name","food_items"])

chain({"cuisine":"Indian food"})

{'cuisine': 'Indian food',
 'restaurant_name': 'Tandoori Nights',
 'food_items': 'Tandoori Nights sounds like a delicious Indian restaurant. Here are some menu item suggestions to get you started:\n\n**Appetizers**\n\n1. Papdi Chaat: Crispy fried flour bread served with yogurt, chickpeas, and tamarind chutney.\n2. Vegetable Samosas: Crunchy pastry filled with spiced potatoes, peas, and onions.\n3. Chicken Tikka: Marinated chicken cooked in a creamy tomato sauce, served with cucumber raita.\n4. Paneer Tikka: Indian cheese marinated in spices and yogurt, grilled to perfection.\n5. Onion Bhajis: Crispy fried onion fritters served with a side of mint chutney.\n\n**Tandoori Specialties**\n\n1. Chicken Tandoori: Classic Indian dish made by marinating chicken in spices and yogurt, then cooking it in a tandoor oven.\n2. Lamb Seekh Kebab: Minced lamb mixed with spices and herbs, grilled to perfection.\n3. Vegetable Tandoori: A colorful mix of marinated vegetables, cooked in a tandoor oven.\n4. 